## Logistic Regression

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#### defines variables

In [2]:
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")

#### Reads from csv file Titanic

In [3]:
data = pd.read_csv('data/Titanic.csv')
data["Age"].fillna(0, inplace=True)
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### gets inputs

In [4]:
def inputs():
    data.replace({"Sex": {"male": 1, "female": 0}}, inplace=True)
    # data = pd.get_dummies(data, columns=['Pclass'])
    train_data = data.sample(frac=0.8, random_state=99)
    test_data = data.loc[~data.index.isin(train_data.index), :]
    X_train = train_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_train = pd.get_dummies(X_train, columns=['Pclass'])
    X_test = test_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_test = pd.get_dummies(X_test, columns=['Pclass'])
    Y_train = train_data.loc[:, ['Survived']]
    Y_test = test_data.loc[:, ['Survived']]
    return X_train, Y_train, X_test, Y_test


X_train, Y_train, X_test, Y_test = inputs()

In [8]:
X_train.head()

,Sex,Age,Pclass_1,Pclass_2,Pclass_3
247,0,24.0,0,1,0
823,0,27.0,0,0,1
566,1,19.0,0,0,1
666,1,25.0,0,1,0
199,0,24.0,0,1,0


In [7]:
Y_train.head()

,Survived
247,1
823,1
566,0
666,0
199,0


#### Inference

In [9]:
def inference(X):
    return tf.sigmoid(tf.matmul(tf.cast(X, tf.float32), W) + b)

#### Loss function using Cross-Entropy

In [22]:
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=tf.matmul(tf.cast(X, tf.float32), W) + b,
        labels=tf.cast(Y, tf.float32)))

In [23]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

#### Evaluating chance of survival ( greater than 0.5 will be choose as survived )

In [26]:
def evaluate(session, X, Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print(session.run(tf.reduce_mean(tf.cast(tf.equal(predicted, tf.cast(Y, tf.float32)), tf.float32))))

In [27]:
with tf.Session() as session:

    tf.global_variables_initializer().run()

    X, Y = X_train, Y_train
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    training_steps = 1000
    for step in range(training_steps):
        session.run([train_op])
        if step % 10 == 0:
            print("loss: ", session.run([total_loss]))

    evaluate(session, X_test, Y_test)
    session.close()

loss:  [0.6949521]
loss:  [0.6800449]
loss:  [0.6736819]
loss:  [0.66905034]
loss:  [0.6651552]
loss:  [0.6616948]
loss:  [0.658539]
loss:  [0.6556146]
loss:  [0.65286905]
loss:  [0.6502597]
loss:  [0.6477513]
loss:  [0.6453161]
loss:  [0.64293337]
loss:  [0.64058995]
loss:  [0.63827795]
loss:  [0.6359934]
loss:  [0.63373494]
loss:  [0.63150215]
loss:  [0.62929523]
loss:  [0.6271144]
loss:  [0.62495995]
loss:  [0.6228317]
loss:  [0.62072974]
loss:  [0.6186541]
loss:  [0.6166044]
loss:  [0.6145805]
loss:  [0.61258197]
loss:  [0.6106084]
loss:  [0.6086597]
loss:  [0.60673547]
loss:  [0.6048358]
loss:  [0.6029599]
loss:  [0.601108]
loss:  [0.59928]
loss:  [0.59747565]
loss:  [0.59569484]
loss:  [0.59393793]
loss:  [0.5922049]
loss:  [0.5904961]
loss:  [0.58881176]
loss:  [0.5871523]
loss:  [0.58551836]
loss:  [0.5839108]
loss:  [0.5823303]
loss:  [0.5807778]
loss:  [0.5792548]
loss:  [0.57776266]
loss:  [0.57630265]
loss:  [0.57487667]
loss:  [0.57348645]
loss:  [0.5721338]
loss:  [0.5708

loss:  [0.56279826]
loss:  [0.56181085]
loss:  [0.5608556]
loss:  [0.559929]
loss:  [0.5590278]
loss:  [0.5581486]
loss:  [0.5572887]
loss:  [0.5564459]
loss:  [0.5556182]
loss:  [0.5548045]
loss:  [0.5540037]
loss:  [0.55321485]
loss:  [0.5524376]
loss:  [0.5516715]
loss:  [0.5509161]
loss:  [0.55017096]
loss:  [0.549436]
loss:  [0.5487107]
loss:  [0.5479952]
loss:  [0.5472889]
loss:  [0.54659176]
loss:  [0.5459034]
loss:  [0.54522365]
loss:  [0.54455256]
loss:  [0.54388946]
loss:  [0.5432346]
loss:  [0.5425875]


loss:  [0.5419481]
loss:  [0.5413163]
loss:  [0.5406918]
loss:  [0.54007435]
loss:  [0.5394641]
loss:  [0.5388607]
loss:  [0.5382639]
loss:  [0.5376737]
loss:  [0.53709006]
loss:  [0.5365128]
loss:  [0.5359416]
loss:  [0.5353766]
loss:  [0.53481734]
loss:  [0.53426397]
loss:  [0.53371626]
0.7752809
